<a href="https://colab.research.google.com/github/mapzine123/myKGAT/blob/main/KGAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 27.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [ ]:
import torch
import csv
import ast

dataset = []

with open('/content/triplets.csv', 'r', encoding='utf-8') as f :
  reader = csv.reader(f)
  for row in reader :
    try :
      triplet = tuple(ast.literal_eval(row[0]))
      dataset.append(triplet)
    except Exception as e :
      print(f"{e}")
print(dataset[0])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GATConv
from dgl import DGLGraph
from dgl import add_self_loop
import csv
import ast

labels = []
dataset = []

with open('../label.csv', 'r', encoding='utf-8') as label :
  reader = csv.reader(label)
  for item in reader :
    labels.append(int(item[0]))

with open('../triplets.csv', 'r', encoding='utf-8') as triplets :
  reader = csv.reader(triplets)
  for item in reader :
    data = ast.literal_eval(item[0])
    dataset.append(data)

class KGAT(nn.Module) :
  def __init__(self, in_feats, hidden_feats, out_feats, num_heads) :
    super(KGAT, self).__init__()
    self.gat1 = GATConv(in_feats, hidden_feats, num_heads)
    self.gat2 = GATConv(hidden_feats * num_heads, out_feats, 1)

  def forward(self, g, inputs) :
    h = self.gat1(g, inputs)
    h = F.elu(h)
    h = h.view(-1, h.size(1) * h.size(2))
    h = self.gat2(g, h)
    return h.mean(1)

def build_graph(triplets) :
  nodes = set()
  for head, relation, tail in triplets :
    if 'uid' in head :
      nodes.add(head['uid'])
      nodes.add(tail['content'])
    elif 'content' in head :
      nodes.add(head['content'])
      nodes.add(tail['hashtag'])

  node_to_id = {node: i for i, node in enumerate(nodes)}
  num_nodes = len(nodes)
  num_relations = len(set(r for _, r, _ in triplets))

  g = DGLGraph()
  g.add_nodes(num_nodes)

  for head, relation, tail in triplets :
        head_id = node_to_id[str(head.get('uid', head.get('content')))]
        tail_id = node_to_id[str(tail.get('content', tail.get('hashtag')))]
        g.add_edges(head_id, tail_id)


  g = add_self_loop(g)  # 그래프에 self-loop 추가

  return g, node_to_id

graph, node_to_id = build_graph(dataset)

num_nodes = graph.number_of_nodes()
in_feats = 10
features = torch.rand((num_nodes, in_feats))

hidden_feats = 8
out_feats = 16
num_heads = 4
model = KGAT(in_feats, hidden_feats, out_feats, num_heads)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

labels = torch.tensor(labels, dtype=torch.long)

for epoch in range(200) :
  model.train()
  logits = model(graph, features)

  num_classes = 16  # 예측할 클래스의 수

  # 레이블 값이 클래스 수를 초과하는 경우 조정합니다.
  adjusted_labels = [label % num_classes for label in labels]
  labels_tensor = torch.tensor(adjusted_labels)
  loss = F.cross_entropy(logits, labels_tensor)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 10 == 0 :
    print(f'Epoch {epoch}, Loss: {loss.item()}')

model.eval()
with torch.no_grad() :
  embeddings = model(graph, features)

with open('embeddings.csv', 'w', newline='') as f :
  writer = csv.writer(f)
  writer.writerow(embeddings)

/usr/local/lib/python3.10/dist-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


Epoch 0, Loss: 2.835883140563965
Epoch 10, Loss: 2.765226125717163
Epoch 20, Loss: 2.754185199737549
Epoch 30, Loss: 2.7514562606811523
Epoch 40, Loss: 2.7497196197509766
Epoch 50, Loss: 2.7487924098968506
Epoch 60, Loss: 2.7481820583343506
Epoch 70, Loss: 2.7476236820220947
Epoch 80, Loss: 2.7470858097076416
Epoch 90, Loss: 2.746532917022705
Epoch 100, Loss: 2.745950698852539
Epoch 110, Loss: 2.745321273803711
Epoch 120, Loss: 2.7446205615997314
Epoch 130, Loss: 2.743833541870117
Epoch 140, Loss: 2.742950916290283
Epoch 150, Loss: 2.741986036300659
Epoch 160, Loss: 2.7409658432006836
Epoch 170, Loss: 2.7398812770843506
Epoch 180, Loss: 2.7387354373931885
Epoch 190, Loss: 2.737518072128296


In [12]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 36.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin